# IVF Index: Making Vector Search Fast

This notebook implements the Inverted File (IVF) Index algorithm for efficient approximate nearest neighbor search.

## The Challenge: Searching Through Millions of Vectors

Vector databases are great for finding similar content, but they face a common problem: as your collection grows, searching gets slower because you need to compare your query against EVERY vector.

Imagine searching through a million vectors - that's a lot of comparisons!

## The Solution: Clustering Vectors

IVF Index solves this by grouping similar vectors into clusters:

1. We divide our vectors into groups (clusters) of similar vectors
2. For each cluster, we choose a representative vector (centroid)
3. When searching, we first find the nearest centroids
4. Then we only search within those few closest clusters

It's like asking "Which section of the library should I look in?" before searching through every book.

In [1]:
#| default_exp ivf_index

In [2]:
#|export

import numpy as np 
from typing import List, Dict, Tuple, Any
from fastkmeans import FastKMeans
from collections import defaultdict
import fastcore.all as fc

from minivecdb.vector_storage import VectorStorage
from minivecdb import gen_emb

## Distance Functions: How We Measure Similarity

To find similar vectors, we need ways to measure how "close" or "far apart" they are. Different distance functions capture different types of similarity:

In [3]:
#|export
def vers(a, b): return 1.0 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8) # 1 - cosine

In [4]:
v1 = np.array([1, 0, 0])       
v2 = np.array([0, 1, 0])
v3 = np.array([-0.5, -0.5, 0])

query = np.array([0.7, 0.7, 0]) 
vers(v1, query), vers(v2, query), vers(v3, query)

(np.float64(0.2928932259563096),
 np.float64(0.2928932259563096),
 np.float64(1.9999999857142858))

In [5]:
#|export
def euclidean(a, b): return np.linalg.norm(a-b)

In [6]:
euclidean(v1, query), euclidean(v2, query), euclidean(v3, query)

(np.float64(0.7615773105863908),
 np.float64(0.7615773105863908),
 np.float64(1.697056274847714))

In [7]:
#|export
def neg_dot(a, b): return -np.dot(a, b) # negative dot

In [8]:
#|export
func_map = {
    'cosine': vers,
    'euclidean': euclidean,
    'dot': neg_dot
}

In [9]:
neg_dot(v1, query), neg_dot(v2, query), neg_dot(v3, query)

(np.float64(-0.7), np.float64(-0.7), np.float64(0.7))

Different distance functions are better for different tasks:

- **Cosine**: Best for semantic similarity when vector size doesn't matter
- **Euclidean**: Best when absolute positions in vector space matter
- **Dot Product**: Good for normalized vectors (all same length)

Our examples show how these measures rank similarity differently for the same vectors.

## Understanding K-means Clustering
K-means clustering is an algorithm that groups similar data points by iteratively assigning points to the nearest of K centers, then updating those centers based on the assigned points. The FastKMeans library we're using optimizes this process through efficient distance calculations and smarter initialization, which helps our IVF Index group similar vectors into clusters – allowing us to search just a few promising clusters rather than the entire database, dramatically improving search speed as our collection grows.

In [10]:
#|export
def run_kmeans(data):
    kmeans = FastKMeans(d=data.shape[1], k=data.shape[0] // 10 + 1,  gpu=False)
    preds = kmeans.fit_predict(data)
    return kmeans.centroids, preds

In [11]:
centroids, preds = run_kmeans(np.random.randn(20, 768))
centroids.shape, preds.shape

((3, 768), (20,))

## Building the Index

The key to the IVF approach is dividing vectors into clusters. We'll use K-means clustering, which:

1. Randomly selects initial cluster centers
2. Assigns each vector to the nearest center
3. Recalculates centers based on assigned vectors
4. Repeats until convergence

Our `build_index` function handles this process:

In [12]:
#|export
def build_index(vs):
    ids, vectors = zip(*[(id, vector) for id, vector, _ in vs.get_all()])
    centroids, preds = run_kmeans(np.array(vectors))
    cluster_ids_map = defaultdict(list)
    id_cluster_map = {}
    for id, cluster_id in zip(ids, preds):
        cluster_ids_map[cluster_id.item()].append(id)
        id_cluster_map[id] = cluster_id.item()
    return cluster_ids_map, id_cluster_map, centroids

In [13]:
vs = VectorStorage.load('data/headline_embeddings.json')

In [14]:
cluster_ids_map, id_cluster_map, centroids = build_index(vs)

## The IVFIndex Class

Now we'll build a complete index that:
1. Stores vectors and cluster assignments
2. Lets us search efficiently
3. Can add new vectors

The key parameters are:
- `nprobe`: How many closest clusters to search (higher = more accurate but slower)
- `distance_fn`: Which distance metric to use (cosine, euclidean, or dot)

## Using the Index

Let's try our index on a real dataset of headline embeddings:

1. Then we build the index from our storage(clustering the vectors)
2. Finally, we search for similar headlines

The stats show us how vectors are distributed across clusters.

In [15]:
#|export
class IVFIndex:
    def __init__(self, storage: VectorStorage, nprobe= 10, distance_fn = "cosine"):
        fc.store_attr()
        self.nprobe = nprobe
        self.distance_fn = distance_fn
        self.is_built = False
        self.dist_func = func_map[distance_fn]
    
    def build(self):
        all_data = self.storage.get_all()
        if not all_data: raise ValueError("Storage is empty, cannot build index")

        self.inverted_lists, self.assignments, self.centroids = build_index(self.storage)
        
        self.is_built = True
        
        sizes = [len(lst) for lst in self.inverted_lists.values()]
        print(f"Average list size: {sum(sizes)/len(sizes):.1f}, Max: {max(sizes)}")
    
    def search(self, query_vector, k = 5):
        if not self.is_built: raise ValueError("Index not built. Call build() first.")
    
        centroid_distances = fc.L(self.centroids.tolist()).enumerate().map(lambda o: (vers(o[1], query_vector).item(), o[0]))
        centroid_distances.sort()

        candidate_ids = centroid_distances[:self.nprobe].itemgot(1).map(lambda o: self.inverted_lists[o]).concat()
        
        if len(candidate_ids) < k:
            for _, cluster_id in centroid_distances[self.nprobe:]:
                candidate_ids.extend(self.inverted_lists[cluster_id])
                if len(candidate_ids) >= k * 2:
                    break

        results = candidate_ids.enumerate().map(lambda o: self.storage.get(o[1])).sorted(key=lambda o: self.dist_func(query_vector, o[0]))
        return results[:k]
    
    
    def add_vectors_bulk(self, vectors, metadata_list):
        ids = fc.L(vectors, metadata_list).zip(lambda o: self.storage.add(o[0], o[1]))
        self.build()
        return ids
    
    def get_stats(self):
        if not self.is_built: return {"built": False}
        
        sizes = [len(lst) for lst in self.inverted_lists.values()]
        return {
            "built": True,
            "n_clusters": len(self.centroids),
            "n_vectors": sum(sizes),
            "avg_cluster_size": sum(sizes) / len(sizes),
            "min_cluster_size": min(sizes),
            "max_cluster_size": max(sizes),
            "empty_clusters": sizes.count(0),
            "nprobe": self.nprobe
        }


In [16]:
index = IVFIndex(vs, nprobe=10, distance_fn="cosine")
index.build()
index.get_stats()

Average list size: 9.8, Max: 28


{'built': True,
 'n_clusters': 39,
 'n_vectors': 383,
 'avg_cluster_size': 9.820512820512821,
 'min_cluster_size': 1,
 'max_cluster_size': 28,
 'empty_clusters': 0,
 'nprobe': 10}

In [17]:
query = "John Korir Claims Victory in Boston Marathon as Evans Lokedi Breaks Women's Course Record"
query_vector = gen_emb.get_embedding(query)
results = index.search(query_vector, k=5)
results.itemgot(1)

(#5) [{'headline': 'Marathon Runner Breaks World Record', 'genre': 'Sports'},{'headline': 'Sprinter Breaks National Record', 'genre': 'Sports'},{'headline': 'Climber Sets New Speed Record', 'genre': 'Sports'},{'headline': 'City Hosts Annual Marathon', 'genre': 'Local News'},{'headline': 'Track Star Qualifies for World Championships', 'genre': 'Sports'}]

## Example Search Results

When we search with a query about marathons and records, we get highly relevant results from our headline dataset - all about sports records and running events.

This demonstrates how semantic search works - finding results based on meaning, not just keywords.

## Adding New Vectors

The index also allows adding new vectors. When adding:
1. We store the vectors in our VectorStorage
2. We rebuild the index to include these new vectors

In a production system, you might use more efficient approaches for updates.

In [18]:
news_headlines = fc.L("Dow Jumps 1,000 Points Tuesday as Markets Rebound from Recent Losses", # Finance
                  "Ronald Acuña Jr. Weeks Away from Return Following Knee Surgery", # Sports
                  "George Clooney Makes Broadway Debut Playing CBS News Legend Edward R. Murrow" # Entertainment
                )

In [19]:
embeddings = news_headlines.map(lambda o: gen_emb.get_embedding(o))

In [20]:
metadata = [{"headline": news_headlines[0], "genre": "finance"}, 
            {"headline": news_headlines[1], "genre": "sports"}, 
            {"headline": news_headlines[2], "genre": "entertainment"}]
index.add_vectors_bulk(embeddings, metadata)

stats = index.get_stats()
print(stats)

Average list size: 9.8, Max: 28
{'built': True, 'n_clusters': 39, 'n_vectors': 383, 'avg_cluster_size': 9.820512820512821, 'min_cluster_size': 1, 'max_cluster_size': 28, 'empty_clusters': 0, 'nprobe': 10}


In [21]:
index.search(query_vector, k=100).itemgot(1).attrgot('headline')

(#100) ['Marathon Runner Breaks World Record','Sprinter Breaks National Record','Climber Sets New Speed Record','City Hosts Annual Marathon','Track Star Qualifies for World Championships','Skier Wins World Championship','Pole Vaulter Sets New Record','Boxer Wins Title in Stunning Knockout','Triathlete Wins National Championship','Swimmer Sets New Olympic Record','Boxer Wins Title in Final Round','Diver Sets New Depth Record','Rowing Team Wins National Title','Skier Wins Gold in World Cup','Cricket Team Secures Historic Victory','Weightlifter Sets New Personal Best','Surfer Wins World Championship','Kayaker Navigates Record-Breaking Rapids','Rugby Team Wins International Tournament','Swimmer Breaks National Record'...]

## Key Takeaways

The IVF Index gives us fast approximate nearest-neighbor search:

- **Speed**: Only search a fraction of vectors
- **Accuracy**: Usually finds the same results as an exhaustive search
- **Trade-offs**: Control speed vs accuracy with `nprobe`

This approach scales well to larger datasets, making vector search practical for real applications.

Next, see 02_gen_embeddings.ipynb to learn how we create vectors from text.

In [24]:
#| hide
import nbdev; nbdev.nbdev_export()